In [1]:
import tensorflow as tf
from konlpy.tag import Twitter
import numpy as np
import gensim

In [2]:
def read_data(filename):    
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]        
        data = data[1:]   # header 제외 #    
    return data 
    
# train_data = read_data('data/ratings_train3.txt')
test_data = read_data('data/ratings_test.txt')
len(test_data)

50000

In [3]:
twitter = Twitter() 

def tokenize(doc):

    return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

# tokens = [tokenize(row[1]) for row in test_data]

In [4]:
model = gensim.models.word2vec.Word2Vec.load('word2vec.model')

In [5]:
len(model.wv.vocab)

def convert2Vec(model, doc):  ## Convert corpus into vectors
    word_vec = []
    word_num = 0
    unk_num = 0
#     print('doc : ', doc)
    for sent in doc:
#         print('sent : ', sent)
        sub = []
        for word in sent:
            word_num += 1
            if(word in model.wv.vocab):
#                 print('word in sent')
                sub.append(model.wv[word])
            else:
                unk_num += 1
#                 print('word not in sent')
                sub.append(np.zeros(300)) ## used for OOV words
        word_vec.append(sub)
    print('word_num :', word_num, 'unk_num :', unk_num)
    return word_vec


In [6]:
# tokens = [[tokenize(row[1]),int(row[2])] for row in test_data if tokenize(row[1]) != []]
tokens = np.array(test_data)
test_X = tokens[:,1]
test_Y = tokens[:,[-1]]
test_X = [tokenize(row) for row in test_X]
test_X = convert2Vec(model, test_X)
maxseq_length = 158

word_num : 734551 unk_num : 20418


In [7]:
def zero_padding(maxseq_length, seq):
    zero_pad = np.zeros((maxseq_length) * 300)
    seq_flat = np.reshape(seq, [-1])
    if(len(seq) > maxseq_length):
        zero_pad[0:] = seq_flat[0:maxseq_length * 300]
    else:
        zero_pad[0:len(seq) * 300] = seq_flat
    return zero_pad
test_X = [zero_padding(maxseq_length, seq) for seq in test_X]
len(test_X[0])

47400

In [8]:
X = tf.placeholder(tf.float32, shape=[None, maxseq_length * 300])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [9]:
save_path = './saved/model-0'

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph(save_path + '.meta')
    saver.restore(sess, save_path)

    graph = tf.get_default_graph()
    
    W = graph.get_tensor_by_name('weight:0')
    b = graph.get_tensor_by_name('bias:0')
    
    hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
    
    acc = sess.run(accuracy , feed_dict={X: test_X, Y: test_Y})
    print('accuracy: ', float(acc))

INFO:tensorflow:Restoring parameters from ./saved/model-0


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float and shape [?,47400]
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[?,47400], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  Fil

In [54]:
accuracy:  0.7890999913215637
accuracy:  0.6700000166893005
